In [1]:
import os
import json
import uuid

def get_content(lines, line_position):
  content_array = []
  for line in lines[line_position:]:
    content_array.append(line)
  # get content
  content = ' '.join(content_array)
  return content

def get_key_value_pairs(lines):
  key_value_pairs = {}
  line_position = 2
  included_keys = ['title']

  try:
    for line in lines[lines.index('---')+1:]:
      line_position += 1
      if line == '---' or line.startswith('#'):
        break
      if ': ' in line:
        key, value = line.split(': ')
        if key in included_keys:
          key_value_pairs[key.strip()] = value.strip()
  except:
    return line_position, key_value_pairs
  return line_position, key_value_pairs

def write_to_jsonl(data, output_json, output_file):
  with open(output_file, 'a') as file:
    file.write(output_json + '\n')

def walk_directory(directory_path, source, output_file):
  json_entries = []
  for root, dirs, files in os.walk(directory_path, source):
    for file in files:
      if file.endswith(".md"):
        data = open(os.path.join(root, file),'r').read()
        lines = data.split('\n')

        line_position, key_value_pairs = get_key_value_pairs(lines)

        # add content to key_value_pairs
        content = get_content(lines, line_position)
        key_value_pairs['content'] = content
        key_value_pairs['source'] = source
        key_value_pairs['file'] = file
        key_value_pairs['_id'] = {"$oid":str(uuid.uuid4())}

        # output json
        output_json = json.dumps(key_value_pairs, indent=4)
        json_entries.append(output_json)
  
  with open(output_file, 'a') as file:
    for entry in json_entries:
      file.write(entry + '\n')

source = "www"
output_file = 'www_output.jsonl'
walk_directory('/Users/kevinbutler/Documents/GitHub/www/content/', source, output_file)

source = "readme_docs"
output_file = 'readme_docs_output.jsonl'
walk_directory('/Users/kevinbutler/Documents/GitHub/readme-docs/v1.0/', source, output_file)

In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client['pinecone']
collection = db['combined']
collection.delete_many({})